In [14]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
# -*- coding: utf-8 -*-
import bs4
import requests, requests.utils, pickle
import requests.packages.urllib3
import sys
requests.packages.urllib3.disable_warnings()
import xlsxwriter
import random
import string
import os
import time
import threading
import copy
import numpy as np

定义基本消息头

In [16]:
headers = {
            'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
            'ContentType': 'text/html; charset=utf-8',
            'Accept-Encoding':'gzip, deflate, sdch',
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language':'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
            'Connection' : 'keep-alive'
        }
cookies = { }
sess = requests.Session()
login_url = 'https://sso.zxxk.com/'   # 学科网登录页面

模拟用户登录

In [17]:
# 进入得登录页面，得到登录所需的参数
def tag_val(tags, key=''):
    value = None
    for tag in tags:

        value = tag.get(key)
        if tag.get('name') == key:
            value = tag.get('value')
            break
    return value

resp = sess.get(login_url, headers=headers)


设置登录所需参数

In [18]:
for k, v in resp.cookies.items():
    cookies[k] = v
    print('cookie key %s value %s' % (k,v))
    
soup = bs4.BeautifulSoup(resp.text, "html.parser")
tags = soup.select('input') 
lt = tag_val(tags,'lt')
execution = tag_val(tags,'execution')
_eventId = tag_val(tags,'_eventId')
rememberMe = tag_val(tags,'rememberMe')
xkw_d = tag_val(tags,'xkw_d')

login_data = {
    'username': '13873153921',
    'password': '123456',
    'lt': lt,
    'execution': execution,
    '_eventId': _eventId,
    'rememberMe': rememberMe,
    'xkw_d': xkw_d
}

print(login_data)

cookie key JSESSIONID value C96FC8A1C2325BD0C68C78A275DD1314
{'username': '13873153921', 'password': '123456', 'lt': 'LT-1088441-NIIYUmfN4McOgVIXSHLM-sso3.zxxk.com', 'execution': 'aace73a6-f931-4f9c-84e8-cd047c35c140_ZXlKaGJHY2lPaUpJVXpVeE1pSjkuUm5wTmIzTnJORUZCUVc1TU1HdFJSMnBaT1ZoNE1sZG5kWFU0YW5neFlqWlpPR2RDZGxweldFdG5XSEJrUzBsUU1rNUZURlJPU21aTFJGZG5VVGszVFZkdWJuQmxObGh2WlhsRWFtOUpLMjlLZWxSSFpHTlNVak41Vms1UWIySTVWRVpDTHpkT2RIQjBTSGROVHpCVk5FOUJla1kxVTJoNmNWVlZiVkUwVVhGcmRsSjBabUpIZDA1VVJWRjRjRm92TldoNFltUXlNV1JVYTNoQ1JHMTRha1Y2ZW5kVVJrWXZjV0UwVjJOUFRFNW5ReTkwV0hseFdFVnpjbk53TlRSa1pXdGhVRGcyWTJSVFdWbFZPV1o0U1hsdE9IZFZTWGhRUmxSSVpWSkRiVzlUWTA5d1dIQlFjR1JCT1ZsNFZqRkhiMGhrY2k5RU1USklVR0o2VUZoTVMzUk5OR1F3VUZWSVMwNVhPVnByY1d4UmMxTTROVTFpUTIxMUwwbzVaWFExZWk4emR6UTRhakZOVFZsRGFFeExPRTU1ZUhJMVN6QjFURmtyY1dkMFVURnBRMVZxUVdsMWQwWXlja3RFUldwRWNTOTZkbTFzUW5CTFNHRnlkR2x2Y1ROc1preGxVVkI0VVU5WE4wZDNUbG9yYVZNNVNWZDFiVmxuTmxSUlpHOHZWbTE2VjA1NFJtMXBaSE5pZHpSWlRTdEZRMVpGYjI1NFVXUXJTMXBDWW10VFpFSnpUR0o0TVd

In [19]:
resp = sess.post('https://sso.zxxk.com/login',data=login_data,headers=headers, allow_redirects=True)

# if resp.status_code == 302:
#     print("登录成功！")
#     for k, v in resp.cookies.items():
#         cookies[k] = v
#         print('cookie key %s value %s' % (k,v))    
#         resp = sess.get(resp.headers['Location'], headers=headers,cookies=resp.cookies)
#         print(resp.text)
# else:
#     print("登录失败！")

In [20]:
print(requests.utils.get_encodings_from_content(resp.text)) #  学科网返回为Gb2312
resp.encoding = 'gb2312'
print(resp.text)

['Gb2312']

<!DOCTYPE html>
<html>
<head>
    <meta charset="Gb2312">
    <!-- 以最高级模式渲染文档 -->
    <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1">
    <!-- 部分国产浏览器默认采用高速模式渲染页面 -->
    <meta name="renderer" content="webkit">
    <link rel="shortcut icon" href="http://www.zxxk.com/favicon.ico">
    <link rel="bookmark" href="http://www.zxxk.com/favicon.ico">

    <title>中学学科网_海量中小学教育资源共享平台、权威教学资源门户网站！</title>
    <meta name="Keywords" content="中学学科网,学科网,2018高考,2018中考,教学资源,教学视频,试卷,试题,课件,PPT,教案,学案,作文,素材,范文,英语,语文,数学,物理,化学,生物,政治,历史,地理,科学,理综,文综">
    <meta name="Description" content="学科网是国内权威中小学教育资源门户网站,拥有试题试卷、课件、教案等教学资源845多万套,内容涵盖小学、初中、高中全部学科,资源多、更新快、内容全、质量高,是老师学生家长优选教育教学资源平台。">
    <link href="http://zxxkstatic.zxxk.com/zxxk/skins/css/common.css" rel="stylesheet" />
    <link href="http://zxxkstatic.zxxk.com/zxxk/skins/css/searchReminder.css" rel="stylesheet" />
    <style>
        .header-box {
            border-bottom: 0;
            padding-bottom: 10px;
            backgr

In [12]:
sess.close()